<a href="https://colab.research.google.com/github/MemphisMeng/BigQuery-AI---Building-the-Future-of-Data/blob/main/Intelligent_Spam_Detective_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## necessary packages
---
It is necessary to make sure the libraries are compatible with each other, therefore, the versions are important.

In [ ]:
# # Install all necessary libraries, including BigQuery client
# !pip install "bigframes==2.9.0"
# !pip install "google-cloud-bigquery[bqstorage,pandas]==3.33.0"
# !pip install "google-cloud-bigquery-storage==2.30.0"
# !pip install "pandas==2.2.2"
# !pip install "protobuf==5.29.1" "rich==13.9.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.1/447.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: bigframes
    Found existing installation: bigframes 2.18.0
    Uninstalling bigframes-2.18.0:
      Successfully uninstalled bigframes-2.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.5/253.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.36.0
    Uninstalling google-cloud-bigquery-3.36.0:
      Successfully uninstalled google-cloud-bigquery-3.36.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery-storage
    Found existing installation: google-cloud-bigquery-storage 2.33.0
    Uninstalling google-cloud-bigquery-storage-2.33.0:
      Successfully uninstalled google-cloud-bigquery-storage-2.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4


## datasets


---
The referenced dataset is publicly accessible on kaggle, credit ML team in UCI.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# input datasets
import pandas as pd
import kagglehub

################################################################################################################
# ONLY RUN THIS CELL WHEN YOU RUN THIS NOTEBOOK FOR THE FIRST TIME IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.#
# kagglehub.login()                                                                                            #
################################################################################################################

dataset_path = 'uciml/sms-spam-collection-dataset'
dataset_directory = kagglehub.dataset_download(dataset_path)
sms_df = pd.read_csv(f"{dataset_directory}/spam.csv", encoding="latin1")

Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.


In [ ]:
# data cleansing
## combine all content from the 2nd to 5th column
sms_df['message'] = None
sms_df['message'].loc[sms_df['Unnamed: 4'].notna()] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str) + ',' + sms_df['Unnamed: 4'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 4'].isna()) & (sms_df['Unnamed: 3'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 3'].isna()) & (sms_df['Unnamed: 2'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str)

sms_df['message'].loc[sms_df['Unnamed: 2'].isna()] = sms_df['v2']

# rename the column to make it meaningful
sms_df.rename(columns={'v1': 'label'}, inplace=True)

# take the necessary columns
sms_df = sms_df[['message', 'label']]

# Model Training

---
The most instinctive approach with the help of BigQuery AI is to ask it whether the give message is a piece of spam or ham, by providing a structured prompt. To do this, a `GeminiTextGenerator` object will be initiated.

In [ ]:
import bigframes
from bigframes.ml.llm import GeminiTextGenerator

PROJECT_ID = "chatbot-278522"  # ⚠️ CHANGE THIS to your actual Google Cloud project ID
bigframes.options.bigquery.project = PROJECT_ID
gemini = GeminiTextGenerator()

Each of the messages in the BigQuery DataFrame is going to be questioned: "`Is the message spam or ham? Please only answer \"spam\" or \"ham\" in ONE word.`"

In [ ]:
gemini_sms_df = gemini.predict(
    sms_df,
    prompt=["Is the message spam or ham? Please only answer \"spam\" or \"ham\" in ONE word.",
            sms_df['message']])

In [ ]:
accuracy = (gemini_sms_df['label'] == gemini_sms_df['ml_generate_text_llm_result'].str.strip()).sum() / len(gemini_sms_df)
print(f"Accuracy of Gemini model: {accuracy:.4f}")

Accuracy of Gemini model: 75.34%


Based on the accuracy performance, there is space for optimization.

In [ ]:
from bigframes.ml.llm import TextEmbeddingGenerator
from bigframes.ml.preprocessing import LabelEncoder

## X


---

The plain text is going to be vectorized.

In [ ]:
embedder = TextEmbeddingGenerator()
embedded_X = embedder.predict(sms_df['message'])

## y
---
As this is a classification challenge, the output column is going to be label encoded.

In [ ]:
from bigframes.ml.preprocessing import LabelEncoder

transformer = LabelEncoder(max_categories=2)
transformed_y = transformer.fit_transform(sms_df['label'])

## Discover the most suitable hyperparameter configuration

In [ ]:
from bigframes.ml.ensemble import XGBClassifier
from bigframes.ml.model_selection import cross_validate

KFold = 3

clf = XGBClassifier()
cv_results = cross_validate(clf, X=embedded_X['ml_generate_embedding_result'], y=transformed_y['labelencoded_label'], cv=KFold)['test_score']

In [ ]:
sum([cv_result['accuracy'] for cv_result in cv_results]) / KFold

0    0.984563
Name: accuracy, dtype: Float64

# So, is that the end?


---

Definitely no! With the detecting tech gets more and more sensitive, the scammers player harder and harder to get. One of their tricks is to put what used to be their plain text messages in an image so that the traditional detection will be circumvented. In this section, I am going to explore the image comprehending and identifying capacities of GCP, with which a multi-modal spam detector will be developed and implemented.

In [ ]:
dataset_path = 'tiharajayawickrama/spam-misleading-images-dataset'
dataset_directory = kagglehub.dataset_download(dataset_path)

100%|██████████| 374M/374M [00:06<00:00, 59.8MB/s]

Extracting files...


In [ ]:
dataset_directory

'/root/.cache/kagglehub/datasets/tiharajayawickrama/spam-misleading-images-dataset/versions/1'